In [1]:
SMALL_SIZE, MEDIUM_SIZE = 9, 12
PROJECT_PATH = "/Users/donghur/Desktop/Research/Murthy/BM_GPU"
# PROJECT_PATH = "/rapids/notebooks/host"

In [2]:
import sys
import numpy as np
from glob import glob
import random, matplotlib
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

sys.path.append(PROJECT_PATH)
from utils.data import Dataset


In [3]:
import utils.figure as ufigure
%load_ext autoreload
%autoreload 1
%aimport utils.data
%aimport utils.figure

In [4]:
Data = Dataset(PROJECT_PATH, 'final')
Data.load_data()

Loading INFO.yaml ...
Finished loading INFO
Loading config.yaml ...


  2%|▏         | 3/141 [00:00<00:05, 24.00it/s]

Finished loading config


100%|██████████| 141/141 [00:06<00:00, 23.23it/s]


In [6]:
# configuration
INFO = Data.info
INFO_values = Data.info_values
config = Data.config
skeleton = config['skeleton']
skeleton_color= config['skeleton_color']

# features
bp = Data.data_obj['bodypoints']
rot_bp = Data.data_obj['rotated_bodypoints']
angles = Data.data_obj['angles']
limbs = Data.data_obj['limbs']
angle_power = Data.data_obj['angle_power']
limb_power = Data.data_obj['limb_power']

# embeddings
all_embed = Data.data_obj['all_embeddings']
all_postural_embed = Data.data_obj['all_postural_embeddings']
all_kinematic_embed = Data.data_obj['all_kinematic_embeddings']

<hr><hr>

In [92]:
bad_fr, bad_bp, bad_ax = np.where(np.isnan(bp))

In [93]:
unique_bad_fr = np.unique(bad_fr)

In [95]:
len(unique_bad_fr)

174253

<hr><hr>

In [21]:
nan_fr, nan_bp, nan_ax = np.where(np.isnan(bp))
inf_fr, inf_bp, inf_ax = np.where(np.isinf(bp))

In [32]:
good_fr, good_ax = np.where(~np.isnan(bp[:,config['bp_center'],0:2]))
good_unique_fr = np.unique(good_fr)

In [31]:

bp[good_unique_fr,:,0:2] -= bp[good_unique_fr,config['bp_center'],0:2][:,np.newaxis,:]

array([   4991,    4992,    4993, ..., 1869436, 1869437, 1869438])

In [35]:
np.where(np.isnan(bp[good_unique_fr,config['bp_center'],0:2]))

(array([], dtype=int64), array([], dtype=int64))

<hr><hr>

In [54]:
num_fr,_,_ = bp.shape
DLC_data = np.copy(bp)


In [71]:
print(DLC_data)
bp_axis = DLC_data[:,config['bp_scale'],0:2]
bad_fr, bad_bp, bad_ax = np.where(np.isnan(bp_axis))
unique_bad_fr = np.unique(bad_fr)

good_idx = np.array([True]*num_fr)
good_idx[unique_bad_fr] = False
good_bp_axis = bp_axis[good_idx,:,:]

x_d = good_bp_axis[0,0] - good_bp_axis[1,0]
y_d = good_bp_axis[0,1] - good_bp_axis[1,1]
dist = np.sqrt(x_d**2+y_d**2)
scale_factor = np.median(dist)
DLC_data[:,:,0:2] /= scale_factor

[[[ 75.1   97.5    1.  ]
  [ 75.44  87.06   1.  ]
  [ 81.56  65.9    1.  ]
  ...
  [ 64.25  46.03   1.  ]
  [ 68.6   62.5    1.  ]
  [ 77.7   66.56   1.  ]]

 [[ 76.8   92.9    1.  ]
  [ 73.2   83.5    1.  ]
  [ 79.5   60.66   1.  ]
  ...
  [ 64.9   33.16   1.  ]
  [ 68.25  56.06   1.  ]
  [ 78.1   61.47   1.  ]]

 [[ 75.5   92.     1.  ]
  [ 72.6   85.1    1.  ]
  [ 76.25  60.72   1.  ]
  ...
  [ 63.06  34.     1.  ]
  [ 65.56  55.56   1.  ]
  [ 75.    61.72   1.  ]]

 ...

 [[ 55.94  73.9    1.  ]
  [ 63.5   74.25   1.  ]
  [ 81.75  76.25   1.  ]
  ...
  [102.    59.34   1.  ]
  [ 91.25  67.3    1.  ]
  [ 82.94  72.06   1.  ]]

 [[ 57.22  74.94   1.  ]
  [ 64.6   70.8    1.  ]
  [ 83.1   75.75   1.  ]
  ...
  [102.7   59.34   1.  ]
  [ 92.6   67.6    1.  ]
  [ 84.44  72.56   1.  ]]

 [[ 55.2   75.06   1.  ]
  [ 62.34  75.1    1.  ]
  [ 80.8   75.8    1.  ]
  ...
  [103.75  58.84   1.  ]
  [ 90.75  66.3    1.  ]
  [ 82.    71.2    1.  ]]]


In [72]:
DLC_data

array([[[16.03 , 20.8  ,  1.   ],
        [16.1  , 18.58 ,  1.   ],
        [17.4  , 14.055,  1.   ],
        ...,
        [13.7  ,  9.82 ,  1.   ],
        [14.64 , 13.336,  1.   ],
        [16.58 , 14.2  ,  1.   ]],

       [[16.39 , 19.81 ,  1.   ],
        [15.62 , 17.81 ,  1.   ],
        [16.95 , 12.94 ,  1.   ],
        ...,
        [13.84 ,  7.074,  1.   ],
        [14.56 , 11.96 ,  1.   ],
        [16.67 , 13.12 ,  1.   ]],

       [[16.11 , 19.62 ,  1.   ],
        [15.49 , 18.16 ,  1.   ],
        [16.27 , 12.95 ,  1.   ],
        ...,
        [13.45 ,  7.254,  1.   ],
        [13.984, 11.85 ,  1.   ],
        [16.   , 13.164,  1.   ]],

       ...,

       [[11.93 , 15.76 ,  1.   ],
        [13.55 , 15.84 ,  1.   ],
        [17.44 , 16.27 ,  1.   ],
        ...,
        [21.77 , 12.66 ,  1.   ],
        [19.47 , 14.36 ,  1.   ],
        [17.69 , 15.375,  1.   ]],

       [[12.2  , 15.984,  1.   ],
        [13.79 , 15.11 ,  1.   ],
        [17.73 , 16.16 ,  1.   ],
        .

In [70]:

np.where(np.isnan(dist))


(array([], dtype=int64),)